# Intervalos multi-dimensionales 

In [412]:
workspace()

In [413]:
using Intervalos

Para resolver problemas en más que una dimensión, será necesario poder calcular un *enclosure* ("encierro") de la imagen de una función sobre un conjunto en varias dimensiones.
Para ello, podemos definir **intervalos multi-dimensionales**, de manera análoga a los intervalos uni-dimensionales que ya conocemos.

[1] ¿Cuál sería una definición matemática razonable de un intervalo multi-dimensional?

Si denotamos un intervalo como

$X_n=[\overline{X}_n, \underline{X}_n]$

Por un intervalo multidimensional, se refiere a una tupla de n intervalos
$X=(X_1, X_2, ..., X_n)$.
Por ejemplo, un intervalo de dos dimensiones

$X = (X_1, X_2) = ([\overline{X}_1, \underline{X}_1],[\overline{X}_2, \underline{X}_2])$

donde, pictográficamente, estaríamos denotando un rectángulo en el plano-$x_1 x_2$: siendo un conjunto de puntos $(x_1, x_2)$,
tal que

$\overline{X}_1 \leq x_1 \leq \underline{X}_1$

$\overline{X}_2 \leq x_2 \leq \underline{X}_2$

[2] ¿Cómo podríamos expresar esto en Julia? Da dos posibilidades.

[Pista: Checa `typealias`]

In [414]:
module MultidimensionalIntervals

  using Intervalos
  import Base.exp, Base.log, Base.sin, Base.cos, Base.tan, Base.length, Base.contains
  export MultiInterval, +, -, *, /, contains, midpoint, Intersection

type MultiInterval
    N::Int
    intervalos::Vector{Interval}    
end

  function MultiInterval(intervalos::Array{Interval,1})
      if length(intervalos)==1 #En caso de ser sólo un intervalo regresa tal intervalo (hace algo similar a typealias)
          return intervalos[1]
      else
          MultiInterval(length(intervalos),intervalos)
      end
  end

  #SUMAS

  function +(X::MultiInterval, Y::MultiInterval)
      if length(X.intervalos)==length(Y.intervalos)
          Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,X.intervalos[i]+Y.intervalos[i])
          end
          return Z
      else
          error("No tienen la misma dimensión")
      end
  end

  function +(X::MultiInterval,N::Real)
      Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,X.intervalos[i]+N)
          end
  end

  function +(N::Real, X::MultiInterval)
      +(X,N)
  end



  #RESTAS

  function -(X::MultiInterval, Y::MultiInterval)
      if length(X.intervalos)==length(Y.intervalos)
          Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,X.intervalos[i]-Y.intervalos[i])
          end
          return Z
      else
          error("No tienen la misma dimensión")
      end
  end

  function +(X::MultiInterval,N::Real)
      Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,X.intervalos[i]-N)
          end
  end

  function -(N::Real, X::MultiInterval)
      -(X,N)
  end

  #Multiplicación

  function *(X::MultiInterval, Y::MultiInterval)
      if length(X.intervalos)==length(Y.intervalos)
          Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,X.intervalos[i]*Y.intervalos[i])
          end
          return Z
      else
          error("No tienen la misma dimensión")
      end
  end

    function *(N::Real,X::MultiInterval)
        Z=MultiInterval(length(X.intervalos),Interval[])
            for i=1:length(X.intervalos)
                push!(Z.intervalos,X.intervalos[i]*N)
            end
        Z
    end

    function *(X::MultiInterval, N::Real)
        *(N,X)
    end


  #División


  function /(X::MultiInterval, Y::MultiInterval)
      if length(X.intervalos)==length(Y.intervalos)
          Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,ExtendedDivision(X.intervalos[i],Y.intervalos[i]))
          end
          return Z
      else
          error("No tienen la misma dimensión")
      end
  end


  function /(N::Real, Y::MultiInterval)
      N_Interval=Interval(N,N)
      Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
          push!(Z.intervalos,ExtendedDivision(N_Interval,Y.intervalos[i]))
          end
          return Z
  end


    function /(Y::MultiInterval, N::Real)
        N_Interval=Interval(N,N)
        Z=MultiInterval(length(X.intervalos),Interval[])
            for i=1:length(X.intervalos)
            push!(Z.intervalos,Y.intervalos[i]/N_Interval)
            end
        return Z
    end


  #Potenciación


  function ^(X::MultiInterval,N::Int)
      Z=MultiInterval(length(X.intervalos),Interval[])
          for i=1:length(X.intervalos)
              push!(Z.intervalos,^(X.intervalos[i],N))
          end
  end

#Funciones varias





end #module

In [415]:
using MultidimensionalIntervals

[3] ¿Cuáles operaciones matemáticas necesitaremos definir sobre los intervalos multi-dimensionales? Impleméntalos en un módulo. ¿Cuál de las dos definiciones resulta más útil en ese respecto?

In [416]:
X1=Interval(-1,1)
X2=Interval(-2,2)
Y1=Interval(1,3)
Y2=Interval(-2,-3)
Y3=Interval(-2.5,-3.3)
X=MultiInterval([X1,X2])
Y=MultiInterval([Y1,Y2])
Yerror=MultiInterval([Y1, Y2, Y3]);

In [380]:
X+Y

MultiInterval(2,[Interval(0,4),Interval(-5,0)])

In [417]:
X-Y

MultiInterval(2,[Interval(-2,-2),Interval(1,4)])

In [337]:
X*Y

MultiInterval(2,[Interval(-3,3),Interval(-6,6)])

In [382]:
X*2

MultiInterval(2,[Interval(-2,2),Interval(-4,4)])

In [388]:
X^2 #NO DA RESULTADO

In [387]:
^(X.intervalos[1],2)

Interval(0.0,1)

In [385]:
X+Yerror

LoadError: No tienen la misma dimensión
while loading In[385], in expression starting on line 1

[4] Considera la función $f: \mathbb{R}^2 \to \mathbb{R}^2$, dada por $f(x,y) = \mathsf{M} \cdot \mathbf{x}$, con $\mathsf{M} = \begin{pmatrix} 2 & 1 \\ 1 & 1 \end{pmatrix}$.

(i) Encuentra a mano la imagen del cuadrado unitario. Encuentra el intervalo multi-dimensional más pequeño que contiene este conjunto.

(ii) Verifica el resultado con tu código.

(iii) ¿Cuál es el problema que ocurre? ¿Cómo lo podríamos resolver? Impleméntalo.

In [421]:
function f(X::MultiInterval)
    x=X.intervalos[1]*2+X.intervalos[2]
    y=X.intervalos[1]+X.intervalos[2]
    return MultiInterval([x,y])
end

f (generic function with 1 method)

In [422]:
X1=Interval(1,2)
X2=Interval(1,2)
X=MultiInterval([X1,X2])
f(X)

MultiInterval(2,[Interval(3,6),Interval(2,4)])

[5] Haz lo mismo para la imagen del cuadrado unitario bajo la función 

$$ \begin{pmatrix} x \\ y \end{pmatrix} \mapsto 
\begin{pmatrix} 1 - ax^2 + y \\ b x \end{pmatrix},$$

con $a = 1.4$ y $b=0.3$ (el famoso "mapeo de Hénon").

In [429]:
function Henon(X::MultiInterval)
    x=1-(X.intervalos[1]^2)*1.4+X.intervalos[2]
    y=X.intervalos[1]*0.3
    return MultiInterval([x,y])
end

Henon (generic function with 1 method)

In [430]:
X1=Interval(1,2)
X2=Interval(1,2)
X=MultiInterval([X1,X2])
Henon(X)

MultiInterval(2,[Interval(1.4,6.6),Interval(0.3,0.6)])